# Content-based filtering with euclidean distance and TFIDF

In [1]:
import pandas as pd 
import numpy as np
#cosine similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df_details=pd.read_csv('games_details_dataset.csv')

In [3]:
user_games = [2620, 2630] #example game ids

In [4]:
for i in range(df_details.shape[0]):
    if df_details.iloc[i]["appid"] in user_games:
        print(df_details.iloc[i]["name"])

Call of Duty
Call of Duty 2


In [5]:
def specific_genre(chosen_genre):
    remove_rows = []
    for i in range(df_details.shape[0]):
        if df_details.iloc[i]["genre_new"] != chosen_genre:
            remove_rows.append(i)
    df = df_details.drop(remove_rows, axis=0)
    user_likes = []
    for i in range(df.shape[0]):
        if user_games.count(df.iloc[i]["appid"])>0:
            user_likes.append(i)
    return df, user_likes

In [6]:
def specific_category(chosen_category):
    remove_rows = []
    for i in range(df_details.shape[0]):
        if df_details.iloc[i]["cat_new"] != chosen_category:
            remove_rows.append(i)
    df = df_details.drop(remove_rows, axis=0)
    user_likes = []
    for i in range(df.shape[0]):
        if user_games.count(df.iloc[i]["appid"])>0:
            user_likes.append(i)
    return df, user_likes

In [7]:
def positive_votes():
    remove_rows = []
    for i in range(df_details.shape[0]):
        if df_details.iloc[i]["rating_sign"] != "positive":
            remove_rows.append(i)
    df = df_details.drop(remove_rows, axis=0)
    user_likes = []
    for i in range(df.shape[0]):
        if user_games.count(df.iloc[i]["appid"])>0:
            user_likes.append(i)
    return df, user_likes

In [8]:
def more_playtime():
    remove_rows = []
    for i in range(df_details.shape[0]):
        if df_details.iloc[i]["playtime"] != "more":
            remove_rows.append(i)
    df = df_details.drop(remove_rows, axis=0)
    user_likes = []
    for i in range(df.shape[0]):
        if user_games.count(df.iloc[i]["appid"])>0:
            user_likes.append(i)
    return df, user_likes

In [9]:
df, user_likes = specific_genre("action")
#df, user_likes = specific_category("multiplayer")
#df, user_likes = positive_votes()
#df, user_likes = more_playtime()

In [10]:
for i in range(df.shape[0]):
    if i in user_likes:
        print(df.iloc[i]["name"])

Call of Duty
Call of Duty 2


## Euclidean distance

### 1st approach

We are using only the attributes: genre, categories and steam spy tags for cosine similarity.

In [11]:
def combined_features_one(row):
    return row['genres'].replace(";"," ").lower()+" "+row['categories'].replace(";"," ").lower()+" "+row['steamspy_tags'].replace(";"," ").lower()
df['cossim'] = df.apply(combined_features_one, axis =1)

In [12]:
cv = TfidfVectorizer()
count_matrix = cv.fit_transform(df['cossim'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.4372291 0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


In [18]:
eucl_dist = euclidean_distances(count_matrix)
print(eucl_dist)

[[0.         0.         0.62053255 ... 1.32772025 1.39331017 1.29111838]
 [0.         0.         0.62053255 ... 1.32772025 1.39331017 1.29111838]
 [0.62053255 0.62053255 0.         ... 1.37369928 1.40702466 1.35698109]
 ...
 [1.32772025 1.32772025 1.37369928 ... 0.         0.6876815  1.09021312]
 [1.39331017 1.39331017 1.40702466 ... 0.6876815  0.         0.97901623]
 [1.29111838 1.29111838 1.35698109 ... 1.09021312 0.97901623 0.        ]]


In [19]:
def find_similar_games(eucl_dist, user_likes, top_k):
    recommended_games = {}
    max_limit_similarity = 0
    max_limit= 0.05
    # repeat until we find enough recommendations
    while len(recommended_games)<20:
        max_limit_similarity+=max_limit
        recommended_games = {}
        for game in user_likes:
            for i in range(len(eucl_dist)):
                if game != i:
                    if eucl_dist[game][i] <= max_limit_similarity:
                        if not (i in user_likes):
                            if i in recommended_games.keys():
                                recommended_games[i][0] +=1
                                if eucl_dist[game][i] < recommended_games[i][1]:
                                    recommended_games[i][1] = eucl_dist[game][i]
                            else:
                                recommended_games[i] = [1, eucl_dist[game][i]]
    recommended_games = {k: v for k, v in sorted(recommended_games.items(), key=lambda item: item[1][0], reverse=True)}
    sort_dict = {}
    lists = []
    list_counter = 0
    frequency = len(eucl_dist) #infinity
    for game, sim_list in recommended_games.items():
        #first list of frequencies
        if frequency == len(eucl_dist):
            frequency = sim_list[0]
            for i in range(frequency):
                lists.append([])
        #sort the list
        if frequency > sim_list[0]:
            frequency = sim_list[0]
            lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1])
            list_counter += 1
            sort_dict.clear()
        sort_dict[game] = sim_list
    lists[list_counter] = sorted(sort_dict.items(), key=lambda item: item[1][1])
    list_counter += 1
    final_sorted_list = []
    for i in range(list_counter):
        final_sorted_list+=lists[i]
    recommended_games = {k: v for k, v in final_sorted_list}
    counter = 0
    result_list = []
    for game, info in recommended_games.items():
        if counter < top_k:
            counter+=1
            result_list.append(df.iloc[game]["appid"])
    return result_list

In [20]:
recommended_games = find_similar_games(eucl_dist, user_likes, 10)
for game in recommended_games:
    for i in range(df.shape[0]):
        if df.iloc[i]["appid"] == game:
            print(df.iloc[i]["name"])
            break

Call of Duty United Offensive
Medal of Honor Airborne
Brothers in Arms Road to Hill 30
Brothers in Arms Hell's Highway
Brothers in Arms Earned in Blood
Ubersoldier II
World War II GI
Enemy Front
Wolfenstein The Old Blood
Delta Force Xtreme 2


### 2nd approach

We will add the attributes publisher and developer and will do the same process .

In [21]:
def combined_features_two(row):
    return row['genres'].replace(";"," ").lower()+" "+row['categories'].replace(";"," ").lower()+" "+row['steamspy_tags'].replace(";"," ").lower()+" "+row['developer'].replace(";"," ").lower()+" "+str(row['publisher']).replace(";"," ").lower()
df['cossim'] = df.apply(combined_features_two, axis =1)

In [22]:
cv = TfidfVectorizer()
count_matrix = cv.fit_transform(df['cossim'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
eucl_dist = euclidean_distances(count_matrix)
print(eucl_dist)

[[0.         0.         0.42059456 ... 1.3900576  1.4085031  1.38048325]
 [0.         0.         0.42059456 ... 1.3900576  1.4085031  1.38048325]
 [0.42059456 0.42059456 0.         ... 1.40298343 1.41228087 1.39856128]
 ...
 [1.3900576  1.3900576  1.40298343 ... 0.         1.3205309  1.36370798]
 [1.4085031  1.4085031  1.41228087 ... 1.3205309  0.         1.35160324]
 [1.38048325 1.38048325 1.39856128 ... 1.36370798 1.35160324 0.        ]]


In [24]:
recommended_games = find_similar_games(eucl_dist, user_likes, 10)
for game in recommended_games:
    for i in range(df.shape[0]):
        if df.iloc[i]["appid"] == game:
            print(df.iloc[i]["name"])
            break

Call of Duty 4 Modern Warfare
Call of Duty Modern Warfare 2
Call of Duty Black Ops III
Homeworld Remastered Collection
Sid Meier's Civilization Beyond Earth
Sid Meier's Civilization V
BioShock Infinite
Call of Duty Infinite Warfare
STAR WARS Jedi Knight II - Jedi Outcast
Call of Duty Ghosts


### 3rd approach

We will add the keywords from the descriptions of the games

In [25]:
def combined_features_three(row):
    return row['genres'].replace(";"," ").lower()+" "+row['categories'].replace(";"," ").lower()+" "+row['steamspy_tags'].replace(";"," ").lower()+" "+row['developer'].replace(";"," ").lower()+" "+str(row['publisher']).replace(";"," ").lower()+" "+' '.join(map(str, row['desc_key_new'])).lower()
df['cossim'] = df.apply(combined_features_three, axis =1)

In [26]:
cv = TfidfVectorizer()
count_matrix = cv.fit_transform(df['cossim'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [27]:
eucl_dist = euclidean_distances(count_matrix)
print(eucl_dist)

[[0.         0.         0.42059456 ... 1.3900576  1.4085031  1.38048325]
 [0.         0.         0.42059456 ... 1.3900576  1.4085031  1.38048325]
 [0.42059456 0.42059456 0.         ... 1.40298343 1.41228087 1.39856128]
 ...
 [1.3900576  1.3900576  1.40298343 ... 0.         1.3205309  1.36370798]
 [1.4085031  1.4085031  1.41228087 ... 1.3205309  0.         1.35160324]
 [1.38048325 1.38048325 1.39856128 ... 1.36370798 1.35160324 0.        ]]


In [28]:
recommended_games = find_similar_games(eucl_dist, user_likes, 10)
for game in recommended_games:
    for i in range(df.shape[0]):
        if df.iloc[i]["appid"] == game:
            print(df.iloc[i]["name"])
            break

Call of Duty 4 Modern Warfare
Call of Duty Modern Warfare 2
Call of Duty Black Ops III
Homeworld Remastered Collection
Sid Meier's Civilization Beyond Earth
Sid Meier's Civilization V
BioShock Infinite
Call of Duty Infinite Warfare
STAR WARS Jedi Knight II - Jedi Outcast
Call of Duty Ghosts


### 4th approach

We will use the descriptions of every game (after cleaning) instead of their keywords.

In [29]:
def combined_features_four(row):
    return row['genres'].replace(";"," ").lower()+" "+row['categories'].replace(";"," ").lower()+" "+row['steamspy_tags'].replace(";"," ").lower()+" "+row['developer'].replace(";"," ").lower()+" "+str(row['publisher']).replace(";"," ").lower()+" "+row['new_description'].lower()
df['cossim'] = df.apply(combined_features_four, axis =1)

In [30]:
cv = TfidfVectorizer()
count_matrix = cv.fit_transform(df['cossim'])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
eucl_dist = euclidean_distances(count_matrix)
print(eucl_dist)

[[0.         1.18171528 1.2551194  ... 1.39539267 1.40751676 1.40092755]
 [1.18171528 0.         1.25528746 ... 1.40028763 1.40964732 1.39981954]
 [1.2551194  1.25528746 0.         ... 1.41172615 1.41358401 1.40941725]
 ...
 [1.39539267 1.40028763 1.41172615 ... 0.         1.37834299 1.39571572]
 [1.40751676 1.40964732 1.41358401 ... 1.37834299 0.         1.38125141]
 [1.40092755 1.39981954 1.40941725 ... 1.39571572 1.38125141 0.        ]]


In [32]:
recommended_games = find_similar_games(eucl_dist, user_likes, 10)
for game in recommended_games:
    for i in range(df.shape[0]):
        if df.iloc[i]["appid"] == game:
            print(df.iloc[i]["name"])
            break

Call of Duty 4 Modern Warfare
Call of Duty World at War
Call of Duty WWII
Call of Duty United Offensive
Call of Duty Black Ops
Call of Duty Ghosts
Call of Duty Black Ops - Mac Edition
Call of Duty Black Ops III
Call of Duty Modern Warfare 2
Sid Meier's Civilization V


The last two approaches has the best results. We put as input 2 call of duty games and we took as a result a list containing call of duty games at the first places of the list.

## Top rated

We will use the ratings, which we calculated before with the Bayesian rating technique.

In [48]:
def TopRated(df):    
    all_games = {}
    top_rated = []
    for i in range(df.shape[0]):
        all_games[i] = df.iloc[i]["rating"]
    top_rated = sorted(all_games.items(), key=lambda item: item[1], reverse=True)
    return top_rated

In [49]:
top_limit = 10
counter = 0
top_rated = TopRated(df_details)
for game, rate in top_rated:
    if counter < top_limit:
        if user_games.count(df_details.iloc[game]["name"])>0:
            continue
        counter+=1
        print(df_details.iloc[game]["name"],"===>",rate)

Portal 2 ===> 9.840933043545515
Factorio ===> 9.783406771972498
The Witcher 3 Wild Hunt ===> 9.753293880115061
Portal ===> 9.73441810716475
Counter-Strike ===> 9.713712651015893
The Binding of Isaac Rebirth ===> 9.710533564390223
RimWorld ===> 9.701293015664707
Terraria ===> 9.691867457316876
Mount & Blade Warband ===> 9.675976490175762
Hotline Miami ===> 9.675027796478476
